In [1]:
# one needs to import those packages which are needed; best to be done at the beginning of the program.
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import scipy as sp
import random as rn
from heapq import nlargest

# some basic settings for plotting figures
import matplotlib.pyplot as plt
%matplotlib inline 
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 32}

plt.rc('font', **font)
import community as community_louvain

In [2]:
G0 = nx.read_weighted_edgelist("4932.protein.links.v11.5.txt",comments="#",nodetype=str)

In [3]:
threshold_score = 700
for edge in G0.edges: 
    weight = list(G0.get_edge_data(edge[0],edge[1]).values())
    if(weight[0] <= threshold_score):
        G0.remove_edge(edge[0],edge[1])

In [4]:
# some basic information
print('number of nodes of G0:',G0.number_of_nodes())
print('number of edges of G0:',G0.number_of_edges())
print('Is the full G0 connected?',nx.connected.is_connected(G0))
print('How many connected subgraphs are there?',nx.connected.number_connected_components(G0))

number of nodes of G0: 6394
number of edges of G0: 120009
Is the full G0 connected? False
How many connected subgraphs are there? 441


In [5]:
#get the largest component
largest_cc = max(nx.connected_components(G0),key=len)
G = G0.subgraph(largest_cc)
print('Type',type(largest_cc))
print('number of nodes of largest connected subgraph of G:',G.number_of_nodes())
print('number of edges of largest connected subgraph of G0:',G.number_of_edges())

Type <class 'set'>
number of nodes of largest connected subgraph of G: 5932
number of edges of largest connected subgraph of G0: 119977


In [6]:
# remove the essential nodes from G0
ess=pd.read_csv("essential_pro.csv",header=None)
ess_pro=pd.Series.to_list(ess[1])
for i in range(len(ess_pro)):
    ess_pro[i]='4932.'+ess_pro[i]
G0.remove_nodes_from(ess_pro)

In [7]:
# new information 53343
print('number of nodes of G0 without essential nodes:',G0.number_of_nodes())
print('number of edges of G0 without essential nodes:',G0.number_of_edges())

number of nodes of G0 without essential nodes: 5098
number of edges of G0 without essential nodes: 53343


In [8]:
# narrow our selection to the proteins connected to ours
nodes = nx.shortest_path(G0,'4932.YKL126W').keys()
G=G0.subgraph(nodes)

In [9]:
# some basic information #3
print('number of nodes of G:',G.number_of_nodes())
print('number of edges of G:',G.number_of_edges())

number of nodes of G: 4639
number of edges of G: 53312


In [10]:
# nx.diameter(G)
#=8

In [23]:
# time to define a parent class of network
class Network:
    R = 1
    N = 10
    MIN_SIZE = 4

    def __init__(self, graph, homologue='4932.YKL126W', partition_method="nx_louvain",partitions=[]):
        self.graph = graph
        self.homologue = homologue
        self.partition_method = partition_method

        self.partitions = partitions
        self.homologue_communities = []
        self.homologue_members={}
        self.central_nodes = []
        self.important_nodes = {}
        self.homologue_index=[]
        self.community_neighbours=[]
        self.adjacent_communities = []
        self.central_nodes_neighbour = [] 
        self.important_nodes_neighbour = {}

        if self.partitions == []:
            self.set_partitions_robust()
        self.set_homologue_communities()
        self.set_central_nodes_robust()
        self.set_important_nodes()

    def set_partitions_robust(self):
        def find_partition(graph, partition_method, s):
            if partition_method == "nx_louvain":
                return nx_comm.louvain_communities(graph, resolution=Network.R, seed=s)

            if partition_method == "other_louvain":
                # some kind of community collection
                return None

        for i in range(Network.N):
            partition = find_partition(self.graph, self.partition_method, i)
            self.partitions.append(partition)

    def set_homologue_communities(self):
        for part in self.partitions:
            for i in range(len(part)):
                if self.homologue in part[i]:
                    sub = self.graph.subgraph(part[i])
                    self.homologue_communities.append(sub)
                    self.homologue_index.append(i)
                    break
    
    def count_homologue_comm_members(self):
        get_subgraph_nodes = lambda x: self.graph.subgraph(x).nodes
        homo_networks = map(get_subgraph_nodes, self.homologue_communities)
        # count the number of subgraph each node occurs in
        flat_comm_nodes = [y for x in homo_networks for y in x]
        for node in list(set(flat_comm_nodes)):
            self.homologue_members[node] = flat_comm_nodes.count(node)

    def set_central_nodes_robust(self):
        def find_central_nodes(community,n=5):
            """return a list of the most significant nodes
            according to three centrality measures"""
            deg = nx.degree_centrality(community)
            bet = nx.betweenness_centrality(community)
            eig = nx.eigenvector_centrality(community)
            top_n_deg = nlargest(n, deg, key=deg.get)
            top_n_bet = nlargest(n, bet, key=bet.get)
            top_n_eig = nlargest(n, eig, key=eig.get)
            return list({*top_n_deg,*top_n_bet,*top_n_eig})

        

        for i in range(Network.N):
            self.central_nodes.append(find_central_nodes(self.homologue_communities[i]))


    def set_c_nodes_neighbour(self):
        def find_c_nodes_neighbour(community, n=3):
            if len(community) < Network.MIN_SIZE: return []
            deg = nx.degree_centrality(community)
            bet = nx.betweenness_centrality(community)

            top_n_deg = nlargest(n, deg, key=deg.get)
            top_n_bet = nlargest(n, bet, key=bet.get)

            return list({*top_n_deg, *top_n_deg})

        for i in range(Network.N):
            neigh_networks = map(self.graph.subgraph, self.adjacent_communities[i])
            cen_neigh = map(find_c_nodes_neighbour, neigh_networks)
            self.central_nodes_neighbour.append(cen_neigh)

    def node_info(self, node, lst):
        spath = nx.shortest_path(self.graph, source=self.homologue, target=node)
        return {
            "times_occurred": lst.count(node),
            "distance": len(spath)
        }

    def set_important_nodes(self):
        # flatten the central nodes list
        flat_central_nodes = sum(self.central_nodes,[])
        for node in set(flat_central_nodes):
            self.important_nodes[node] = self.node_info(node, flat_central_nodes)

    def set_important_nodes_neighbour(self):
        # flatten the central nodes list
        flat_central_nodes_1 = sum( self.central_nodes_neighbou,[])
        flat_central_nodes_2 = sum( flat_central_nodes_1 ,[])
        for node in set(flat_central_nodes_2):
            self.important_nodes_neighbour[node] = self.node_info(node, flat_central_nodes_2)

    def find_neighbours(self):
        for comm in self.homologue_communities:
            nodes = comm.nodes
            neighs = set()
            for n in nodes:
                neighs.update([*self.graph.neighbors(n)])
            self.community_neighbours.append(neighs)

    def set_neighbour_communities(self):
        a = self.partitions.copy()
        for i, part in enumerate(a):
            del part[self.homologue_index[i]]
            neighs = self.community_neighbours[i]
            # all communities containing a neighbouring element
            nei_comm = [comm for comm in part if set(comm) & set(neighs) != set()]
            self.adjacent_communities.append(nei_comm)

    def get_partitions(self):
        return self.partitions

    def get_homologue_communities(self):
        return self.homologue_communities

    def get_central_nodes(self):
        return self.central_nodes
    
    def get_important_nodes(self):
        return self.important_nodes

In [24]:
akt2 = Network(G, '4932.YKL126W')

In [25]:
akt2.get_important_nodes()

{'4932.YBL016W': {'times_occurred': 9, 'distance': 4},
 '4932.YLR433C': {'times_occurred': 7, 'distance': 3},
 '4932.YDR477W': {'times_occurred': 9, 'distance': 3},
 '4932.YLR362W': {'times_occurred': 9, 'distance': 3},
 '4932.YOL012C': {'times_occurred': 1, 'distance': 3},
 '4932.YJR066W': {'times_occurred': 9, 'distance': 2},
 '4932.YMR190C': {'times_occurred': 1, 'distance': 4},
 '4932.YGR040W': {'times_occurred': 9, 'distance': 4},
 '4932.YHL007C': {'times_occurred': 4, 'distance': 3},
 '4932.YBR010W': {'times_occurred': 1, 'distance': 3},
 '4932.YNL098C': {'times_occurred': 8, 'distance': 3},
 '4932.YLR113W': {'times_occurred': 9, 'distance': 3},
 '4932.YHR030C': {'times_occurred': 9, 'distance': 3},
 '4932.YMR307W': {'times_occurred': 4, 'distance': 3},
 '4932.YKL113C': {'times_occurred': 1, 'distance': 4},
 '4932.YOR033C': {'times_occurred': 1, 'distance': 4},
 '4932.YML032C': {'times_occurred': 1, 'distance': 4},
 '4932.YER095W': {'times_occurred': 1, 'distance': 3},
 '4932.YHL

In [26]:
# for part in sorted(akt2.get_partitions(), key=lambda x:len(x)):
#     print(len(part), sorted([len(i) for i in part],reverse=True))

In [27]:
# subnet_nodes = [*{
#     node: akt2.homologue_members[node]
#         for node in akt2.homologue_members \
#             if akt2.homologue_members[node] >= 7
# }.keys()]
# subnet = G.subgraph(subnet_nodes)
# nx.draw(subnet)

In [28]:
# akt2.important_nodes_neighbour

In [29]:
akt2.homologue_index

[6, 8, 16, 14, 15, 10, 14, 17, 12, 16]

In [30]:
# akt2.partitions

In [31]:
# len([*G.neighbors("4932.YKL126W")])

In [32]:
# nodes_n = akt2.adjacent_communities[0]
# "4932.YKL126W" in [y for x in nodes_n for y in x]

In [33]:
# len(nodes_n)

In [34]:
# a = akt2.partitions[0]
# len(a)